In [1]:
import torch, time

In [12]:
A = torch.rand(10_000, 10_000).T   # default is float32, so this is about 400 MB

In [15]:
start = time.time()
A.sum(dim=0)    # 0 is down, so this is summing over columns
end = time.time()
(end-start)*1000   # milliseconds

53.925514221191406

In [16]:
start = time.time()
A.sum(dim=1)    # 1 is across, so this is summing over rows
end = time.time()
(end-start)*1000   # milliseconds

124.97138977050781

# PyArrow

In [2]:
#!wget https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.zip

In [4]:
#!unzip hdma-wi-2021.zip

In [5]:
!ls -lah

total 188M
drwxrwxr-x 3 trh trh 4.0K Sep 29 18:29 .
drwxrwxr-x 5 trh trh 4.0K Sep 26 20:28 ..
drwxrwxr-x 2 trh trh 4.0K Sep 29 18:13 .ipynb_checkpoints
-rw-rw-r-- 1 trh trh  278 Sep 26 20:28 Dockerfile
-rw-rw-r-- 1 trh trh 2.5K Sep 29 18:29 demo.ipynb
-rw-r----- 1 trh trh 167M Nov  1  2022 hdma-wi-2021.csv
-rw-rw-r-- 1 trh trh  21M Jan  5  2023 hdma-wi-2021.zip


In [8]:
import time
import pandas as pd

In [9]:
start = time.time()
pd.read_csv("hdma-wi-2021.csv")
end = time.time()
end-start # seconds

/tmp/ipykernel_4381/3879644429.py:2: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv("hdma-wi-2021.csv")


6.545098543167114

In [3]:
import pyarrow.csv
import time

In [4]:
start = time.time()
tbl = pyarrow.csv.read_csv("hdma-wi-2021.csv")
end = time.time()

In [5]:
end-start

1.98746657371521

In [7]:
start = time.time()
df = tbl.to_pandas()
end = time.time()
end-start

1.2285869121551514

In [9]:
import pyarrow.compute as pc

In [12]:
pc.utf8_lower(tbl["lei"]).to_pandas()

0         54930034mnpilhp25h80
1         54930034mnpilhp25h80
2         54930034mnpilhp25h80
3         54930034mnpilhp25h80
4         54930034mnpilhp25h80
                  ...         
447362    54930034mnpilhp25h80
447363    54930034mnpilhp25h80
447364    54930034mnpilhp25h80
447365    54930034mnpilhp25h80
447366    54930034mnpilhp25h80
Length: 447367, dtype: object

In [18]:
pc.mean(tbl["income"].drop_null()).as_py()

377.5220353645974

In [21]:
tbl[:10].to_pandas()

,activity_year,lei,derived_msa-md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,denial_reason-2,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,2021,54930034MNPILHP25H80,99999,WI,55027,55027961800,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4196,3.67,69600,108,1422,1839,57
1,2021,54930034MNPILHP25H80,99999,WI,55001,55001950501,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,1511,5.43,69600,65,541,1966,33
2,2021,54930034MNPILHP25H80,99999,WI,55013,55013970400,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,3895,9.63,69600,80,1685,5859,35
3,2021,54930034MNPILHP25H80,29404,WI,55059,55059002000,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,5561,9.15,102500,106,1851,2208,30
4,2021,54930034MNPILHP25H80,11540,WI,55087,55087013300,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Joint,...,NaN,NaN,NaN,7248,5.22,85600,111,1939,2351,14
5,2021,54930034MNPILHP25H80,24580,WI,55061,55061960500,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,6136,4.56,80100,104,2136,2627,53
6,2021,54930034MNPILHP25H80,43100,WI,55117,55117000100,NC,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,3803,16.49,79600,97,1011,1483,76
7,2021,54930034MNPILHP25H80,43100,WI,55117,55117000100,C,Conventional:Subordinate Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,3803,16.49,79600,97,1011,1483,76
8,2021,54930034MNPILHP25H80,99999,WI,55103,55103970100,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,2909,3.71,69600,84,969,1490,67
9,2021,549300FQ2SN6TRRGB032,99999,WI,55127,55127001501,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,3160,15.06,69600,126,983,2219,22
